In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from numpy import mean
from numpy import std
from math import sqrt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, KBinsDiscretizer
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
import math
from parseit.data import load_pickle

# Load pickle
pickle_df_with_features = load_pickle(f"99-with-features.p")

In [143]:
df = pickle_df_with_features.copy()


k = 50
bins = 10
test_size = 0.3
X = dfwf[["top-cos-sim", "cos-sim", "tfidf-mean", "wc", "sw", "bw", "smil+", "smil-", "smil&"]]
#X = dfwf[["top-cos-sim", "wc", "bw", "smil+"]]
y = dfwf[["label"]]


# Bin all of y first
# uniform = All bins in each feature have identical widths.
# quantile = All bins in each feature have the same number of points.
# kmeans = Values in each bin have the same nearest center of a 1D k-means cluster.
#est = KBinsDiscretizer(n_bins=bins, encode="ordinal", strategy="uniform") #
#est.fit(y)
#y = pd.DataFrame(data=est.transform(y), index=y.index)

#drop = y[y == 0][100:]
#y = y.drop(index=drop.index)
#X = X.drop(index=drop.index)
#display(y)

X_unscaled_train, X_unscaled_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

# All variables must be in the same scale: normalization or min-max-scaler
# How to scale so that each distance is meaningful: https://medium.com/analytics-vidhya/why-is-scaling-required-in-knn-and-k-means-8129e4d88ed7
scaler = StandardScaler()
#scaler = MinMaxScaler()
scaler.fit(X_unscaled_train)
X_train = scaler.transform(X_unscaled_train)


# Try with SKF
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
knn = KNeighborsClassifier(n_neighbors=k)
n_scores = cross_val_score(knn, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score="raise")
# report model performance
print("Accuracy (SFK): %.2f%% (%.2f%%)\n" % (mean(n_scores)*100, std(n_scores)*100))


# Try without SKF
knn = KNeighborsClassifier(n_neighbors=k)
clf = knn.fit(X_train, y_train)
y_pred = clf.predict(scaler.transform(X_unscaled_valid))
#print(y_pred)
accuracy = metrics.accuracy_score(y_valid, y_pred, normalize=False)
accuracy_norm = metrics.accuracy_score(y_valid, y_pred, normalize=True)
print(f'Accuracy: {"%.2f" % (accuracy_norm * 100)}% ({accuracy} comments of {math.floor(len(y.index)*test_size)}) for k={k}\n')

# Try with LinearSVC
svc = LinearSVC(random_state=0, tol=1e-05)
clf = svc.fit(X_train, y_train)
# Scale X_valid after fitting the model: https://datascience.stackexchange.com/questions/38395/standardscaler-before-and-after-splitting-data
y_pred = clf.predict(scaler.transform(X_unscaled_valid))
print(y_pred)
accuracy = metrics.accuracy_score(y_valid, y_pred, normalize=False)
accuracy_norm = metrics.accuracy_score(y_valid, y_pred, normalize=True)
print(f'Linear SVC Accuracy: {"%.2f" % (accuracy_norm * 100)}% ({accuracy} comments of {math.floor(len(y.index)*test_size)})')

# Try different ranges for k
k_range = range(1, k, math.floor(k/10))
for kk in k_range:
    knn = KNeighborsClassifier(n_neighbors=kk)
    clf = knn.fit(X_train, y_train)
    y_pred = clf.predict(scaler.transform(X_unscaled_valid))
    accuracy = metrics.accuracy_score(y_valid, y_pred, normalize=False)
    accuracy_norm = metrics.accuracy_score(y_valid, y_pred, normalize=True)
    print(f'Accuracy: {"%.2f" % (accuracy_norm * 100)}% ({accuracy} comments of {math.floor(len(y.index)*test_size)}) for k={kk}')
    


display(y.value_counts())

Accuracy (SFK): 9.40% (1.28%)

Accuracy: 7.45% (14 comments of 187) for k=50

[    5     2     2     5     2     2     5     5     2     2   118     2
   194     2     2     2     2   272   163     2   596     2     5     2
     5   194     2     2     2     2  5220   230     5   272     2   194
     2     2     2   596     2     2     2     2     2   163     2     2
     2     2     2     2     2     2    19     3   163     2     2     2
  8718     2     2     2     2   111   194  9216     2     2     5     4
     2    18     7     2     2     2     2     2     5     2    37    75
     2     2     2    75     2    62     2   194     2     2     2     2
 13588  3050     2     5     2     2   118     2     2     2   410     2
     2     2     2     2     2   111    75     2     2   111     2     2
     2   163     2     2     2  5220     2     2     2     5     2     5
     2     2     2     2     2     2     2    73     5     2 34290     2
     2     2     2   202     3    75     5    

label
2        61
3        35
5        23
4        15
6        13
7         8
12        7
18        7
9         6
26        5
15        5
14        5
8         5
10        5
20        5
62        4
78        4
36        4
17        4
41        4
60        3
27        3
45        3
63        3
336       3
136       3
21        3
24        3
23        3
19        3
11        3
150       2
163       2
66        2
53        2
167       2
370       2
88        2
58        2
238       2
185       2
54        2
190       2
52        2
84        2
13        2
16        2
127       2
28        2
34        2
206       2
37        2
40        2
106       2
105       2
236       2
83        2
295       1
290       1
286       1
298       1
285       1
284       1
281       1
293       1
63821     1
280       1
307       1
224       1
227       1
229       1
230       1
231       1
234       1
239       1
277       1
253       1
255       1
258       1
259       1
272       1
275       1
302       